In [1]:
import os
import pandas as pd
import datetime
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import numpy as np
import scipy 
import math
import h5py
import time
import copy
import sys
import feather
from sklearn.metrics import mean_absolute_error
from numpy.random import seed
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
seed(123)
from tensorflow import set_random_seed
set_random_seed(456)
np.set_printoptions(threshold=sys.maxsize)

In [30]:
view_df = feather.read_dataframe("/home/whsu014/data/view_1hot_nona_len29_float_age_impute_and_True_HG_100096inds.feather")
print(view_df.columns)
print(view_df.shape)

Index(['VSIMPLE_INDEX_MASTER', 'QUARTER', 'AGE', 'SEX', 'NZDEP', 'ETHN_1',
       'ETHN_2', 'ETHN_3', 'ETHN_4', 'ETHN_5',
       ...
       'PT_DIABETES_YR', 'PT_ATRIAL_FIBRILLATION', 'PT_IMP_FATAL_CVD',
       'TRUE_HDL', 'TRUE_LDL', 'TRUE_TRI', 'TRUE_TCL', 'TRUE_TCHDL',
       'TRUE_HBA1C', 'TRUE_EGFR'],
      dtype='object', length=191)
(2902784, 191)


In [3]:
print('AGE:', view_df.columns.get_loc('AGE'))
print('NZDEP:', view_df.columns.get_loc('NZDEP'))
print('ETHN_1:', view_df.columns.get_loc('ETHN_1'))
print('ETHN_5:', view_df.columns.get_loc('ETHN_5'))
print('TEST:', view_df.columns.get_loc('TEST'))
print('HDL:', view_df.columns.get_loc('HDL'))
print('LDL:', view_df.columns.get_loc('LDL'))
print('TRI:', view_df.columns.get_loc('TRI'))
print('TCL:', view_df.columns.get_loc('TCL'))
print('TCHDL:', view_df.columns.get_loc('TCHDL'))
print('TESTED:', view_df.columns.get_loc('TESTED'))
print('DIED:', view_df.columns.get_loc('DIED'))
print('############# PHARMS ##############')
print('LL_SIMVASTATIN:', view_df.columns.get_loc('LL_SIMVASTATIN'))
print('LL_PDC:', view_df.columns.get_loc('LL_PDC'))
print('CVD_FELODIPINE:', view_df.columns.get_loc('CVD_FELODIPINE'))
print('CVD_HYDROCHLOROTHIAZIDE:', view_df.columns.get_loc('CVD_HYDROCHLOROTHIAZIDE'))
print('OTHER_TENOXICAM:', view_df.columns.get_loc('OTHER_TENOXICAM'))
print('OTHER_VILDAGLIPTIN:', view_df.columns.get_loc('OTHER_VILDAGLIPTIN'))
print('############# NMDS ##############')
print('NUMBER_OF_DAYS:', view_df.columns.get_loc('NUMBER_OF_DAYS'))
print('hx_broad_cvd:', view_df.columns.get_loc('hx_broad_cvd'))
print('out_atrial_fibrillation:', view_df.columns.get_loc('out_atrial_fibrillation'))
print('############# HbA1c, eGFR and PT ##############')
print('HBA1C:', view_df.columns.get_loc('HBA1C'))
print('TEST_HBA1C:', view_df.columns.get_loc('TEST_HBA1C'))
print('TESTED_HBA1c:', view_df.columns.get_loc('TESTED_HBA1C'))
print('EGFR:', view_df.columns.get_loc('EGFR'))
print('TEST_GFR:',view_df.columns.get_loc('TEST_EGFR'))
print('TESTED_GFR:', view_df.columns.get_loc('TESTED_EGFR'))
print('PT_SBP:', view_df.columns.get_loc('PT_SBP'))
print('PT_DIABETES_YR:', view_df.columns.get_loc('PT_DIABETES_YR'))
print('PT_ATRIAL_FIBRILLATION:', view_df.columns.get_loc('PT_ATRIAL_FIBRILLATION'))
print('PT_IMP_FATAL_CVD:', view_df.columns.get_loc('PT_IMP_FATAL_CVD'))

AGE: 2
NZDEP: 4
ETHN_1: 5
ETHN_5: 9
TEST: 10
HDL: 11
LDL: 12
TRI: 13
TCL: 14
TCHDL: 15
TESTED: 16
DIED: 17
############# PHARMS ##############
LL_SIMVASTATIN: 18
LL_PDC: 29
CVD_FELODIPINE: 30
CVD_HYDROCHLOROTHIAZIDE: 83
OTHER_TENOXICAM: 84
OTHER_VILDAGLIPTIN: 121
############# NMDS ##############
NUMBER_OF_DAYS: 122
hx_broad_cvd: 124
out_atrial_fibrillation: 164
############# HbA1c, eGFR and PT ##############
HBA1C: 165
TEST_HBA1C: 166
TESTED_HBA1c: 167
EGFR: 168
TEST_GFR: 169
TESTED_GFR: 170
PT_SBP: 171
PT_DIABETES_YR: 181
PT_ATRIAL_FIBRILLATION: 182
PT_IMP_FATAL_CVD: 183


In [4]:
print(view_df.shape)
view_v = copy.deepcopy(view_df.values)
print(view_v.shape)

(2902784, 191)
(2902784, 191)


In [5]:
died_ids = []

num_of_ids = view_v.shape[0]//29
print("Number of inds:", num_of_ids)

for i in range(num_of_ids):
    ind_v = view_v[(i*29):((i+1)*29), :]
    if sum(ind_v[:25, 17]) > 0:
        died_ids.append(ind_v[0, 0])
print("Number of inds died by the 25th quarter (len 29):",
      len(died_ids))

Number of inds: 100096
Number of inds died by the 25th quarter (len 29): 4897


In [6]:
# removed individuals who died in new obs window
view_v = view_df[~view_df['VSIMPLE_INDEX_MASTER']\
                .isin(died_ids)].values
print("Number of inds:", view_v.shape[0]//29)

Number of inds: 95199


In [7]:
train_v = view_v[:(((95000//10)*9)*29), :]
test_v = view_v[(((95000//10)*9)*29):, :]
print("Number of inds in train set:", train_v.shape[0]/29)
print("number of inds in test set:", test_v.shape[0]/29)

Number of inds in train set: 85500.0
number of inds in test set: 9699.0


# Set up test set, flatten TestSafe vectors

In [8]:
# remove CVD_ISOSORBRIDE_DINITRATE
print(test_v.shape)
test_df = pd.DataFrame(test_v)
test_df.columns = list(view_df.columns)
print(test_df['CVD_ISOSORBRIDE_DINITRATE'].unique())
test_df = test_df.drop(['CVD_ISOSORBRIDE_DINITRATE'], axis=1)
print(test_df.shape)

(281271, 191)
[0.0]
(281271, 190)


In [9]:
print('TEST:', test_df.columns.get_loc('TEST'))
print('HDL:', test_df.columns.get_loc('HDL'))
print('LDL:', test_df.columns.get_loc('LDL'))
print('TRI:', test_df.columns.get_loc('TRI'))
print('TCL:', test_df.columns.get_loc('TCL'))
print('TCHDL:', test_df.columns.get_loc('TCHDL'))
print('TESTED:', test_df.columns.get_loc('TESTED'))
print('DIED:', test_df.columns.get_loc('DIED'))
print('TRUE_HDL:', test_df.columns.get_loc('TRUE_HDL'))
print('TRUE_LDL:', test_df.columns.get_loc('TRUE_LDL'))
print('TRUE_TRI:', test_df.columns.get_loc('TRUE_TRI'))
print('TRUE_TCL:', test_df.columns.get_loc('TRUE_TCL'))
print('TRUE_TCHDL:', test_df.columns.get_loc('TRUE_TCHDL'))

TEST: 10
HDL: 11
LDL: 12
TRI: 13
TCL: 14
TCHDL: 15
TESTED: 16
DIED: 17
TRUE_HDL: 183
TRUE_LDL: 184
TRUE_TRI: 185
TRUE_TCL: 186
TRUE_TCHDL: 187


In [10]:
# Interpolated cholesterol vectors
# allow observation data to peak
# forward in time.
# Remedy by:
# flatten Cholesterol vectors
# after the last test within
# the observation window (len 29) using 
# TRUE cholesterol values
test_v = copy.deepcopy(test_df.values)

number_of_inds = test_v.shape[0]//29
print(number_of_inds)
measure_not_in_last_test = 0
ids_of_measure_not_in_last_test = []
missing_test = 0
ids_of_missing_test = []
value_indicating_test = max(np.unique(test_v[:, 10]))
value_indicating_no_test = min(np.unique(test_v[:, 10]))
print("V test:", value_indicating_test)
print("V no test :", value_indicating_no_test)
#view_1hot_test_v = np.empty([view_1hot_v.shape[0],\
#                                     view_1hot_v.shape[1]], dtype="object")
start_time = time.time()
for i in range(number_of_inds):
    ind_v = copy.deepcopy(test_v[(i*29):((i+1)*29), :])
    obs_test = ind_v[:25, 10]
    test_indices_list = [idx for idx, v in enumerate(obs_test) if v==value_indicating_test]
    for chol_idx in range(0, 5): # TRUE Chol 183-187 
        last_idx = -1
        last_test_idx = test_indices_list[last_idx]
        chol_str = ind_v[last_test_idx, (chol_idx+183)]
        ##############################################
        # if the last test didn't test for this chol
        ##############################################
        try:
            while len(chol_str) <= 2:
                measure_not_in_last_test += 1
                ids_of_measure_not_in_last_test.append(ind_v[0, 0])
                last_idx = last_idx - 1
                ###### could cause IndexError ######
                last_test_idx = test_indices_list[last_idx]
                ####################################
                chol_str = ind_v[last_test_idx, (chol_idx+183)]
        except IndexError:
            missing_test += 1
            ids_of_missing_test.append(ind_v[0, 0])
            continue
        chol_float = float(chol_str[1:-1].split(", ")[-1])
        ind_v[last_test_idx:, (chol_idx+11)] = chol_float
    test_v[(i*29):((i+1)*29), :] = copy.deepcopy(ind_v)
    if i%1000 == 0:
        print("Individual:", i)
        print(time.time() - start_time)
print("Total time:", time.time() - start_time)
print("Measure not in last test count:", measure_not_in_last_test)
print("IDs :", len(np.unique(ids_of_measure_not_in_last_test)))
print("Missing Test:", missing_test)
print("IDs of missing Test:", len(np.unique(ids_of_missing_test)))
print(test_v.shape)

9699
V test: 1
V no test : 0
Individual: 0
2.743359088897705
Individual: 1000
8.062700510025024
Individual: 2000
13.300366163253784
Individual: 3000
18.73568558692932
Individual: 4000
23.98189425468445
Individual: 5000
29.2449893951416
Individual: 6000
34.487430810928345
Individual: 7000
39.71019744873047
Individual: 8000
44.91251230239868
Individual: 9000
50.180970191955566
Total time: 53.81725215911865
Measure not in last test count: 571
IDs : 274
Missing Test: 0
IDs of missing Test: 0
(281271, 190)


In [14]:
# data check
#print(view_df['CVD_ISOSORBRIDE_DINITRATE'].unique())
#view_df = view_df.drop(['CVD_ISOSORBRIDE_DINITRATE'], axis=1)

#test_df = pd.DataFrame(test_v)
#test_df.columns = list(view_df.columns)
#idx = 10000
#test_df.iloc[(idx*29):((idx+1)*29), 
#             [0,1,10,11,12,13,14,15,183,184,185,186,187]]
ind_id = ids_of_measure_not_in_last_test[10]
test_df[test_df.iloc[:, 0]==ind_id]\
.iloc[:, [0,1,10,11,12,13,14,15,183,184,185,186,187]]

,VSIMPLE_INDEX_MASTER,QUARTER,TEST,HDL,LDL,TRI,TCL,TCHDL,TRUE_HDL,TRUE_LDL,TRUE_TRI,TRUE_TCL,TRUE_TCHDL
6380,53214,0,0,1.1,2.6,2.1,4.6,4.3,[],[],[],[],[]
6381,53214,1,0,1.1,2.6,2.1,4.6,4.3,[],[],[],[],[]
6382,53214,2,0,1.1,2.6,2.1,4.6,4.3,[],[],[],[],[]
6383,53214,3,0,1.1,2.6,2.1,4.6,4.3,[],[],[],[],[]
6384,53214,4,0,1.1,2.6,2.1,4.6,4.3,[],[],[],[],[]
6385,53214,5,1,1.10576,2.63289,2.14934,4.66578,4.32467,[1.1],[2.6],[2.1],[4.6],[4.3]
6386,53214,6,0,1.12247,2.72838,2.29257,4.85676,4.39629,[],[],[],[],[]
6387,53214,7,0,1.13918,2.82387,2.43581,5.04775,4.4679,[],[],[],[],[]
6388,53214,8,0,1.15589,2.91936,2.57905,5.23873,4.53952,[],[],[],[],[]
6389,53214,9,1,1.16188,2.97971,2.72029,5.37971,4.62029,[1.17],[3.0],[2.7],[5.4],[4.6]


In [15]:
print('HBA1C:', test_df.columns.get_loc('HBA1C'))
print('TEST_HBA1C:', test_df.columns.get_loc('TEST_HBA1C'))
print('TESTED_HBA1c:', test_df.columns.get_loc('TESTED_HBA1C'))
print('EGFR:', test_df.columns.get_loc('EGFR'))
print('TEST_EGFR:', test_df.columns.get_loc('TEST_EGFR'))
print('TESTED_EGFR:', test_df.columns.get_loc('TESTED_EGFR'))
print('TRUE_HBA1C:', test_df.columns.get_loc('TRUE_HBA1C'))
print('TRUE_EGFR:', test_df.columns.get_loc('TRUE_EGFR'))

HBA1C: 164
TEST_HBA1C: 165
TESTED_HBA1c: 166
EGFR: 167
TEST_EGFR: 168
TESTED_EGFR: 169
TRUE_HBA1C: 188
TRUE_EGFR: 189


In [16]:
####################################
# flatten HbA1c and GFR from last
# test in obs window for test set
####################################
print(test_v.shape)
num_of_inds = test_v.shape[0]//29
print("Number of inds:", num_of_inds)

H_in_obs_wind = []
H_bf_obs_wind = []
H_other = []

G_in_obs_wind = []
G_bf_obs_wind = []
G_other = []

start_time = time.time()
for i in range(num_of_inds):
    ind_v = copy.deepcopy(test_v[(i*29):((i+1)*29), :])
    if sum(ind_v[:25, 165]) > 0: # HbA1c tested in obs window
        last_test_idx = 25 - 1- (list(ind_v[:25, 165])[::-1].index(1)) # 1st HbA1c test in obs window
        true_HbA1c_str = ind_v[last_test_idx, 188]
        true_HbA1c_float = float(true_HbA1c_str[1:-1].split(',')[-1]) #flatten for test set
        ind_v[last_test_idx:, 164] = true_HbA1c_float
        H_in_obs_wind.append(ind_v[0, 0])
    elif sum(ind_v[:25, 166]) > 0: # HbA1c tested prior to obs window and not in obs window
        true_HbA1c_str = ind_v[0, 188]
        true_HbA1c_float = float(true_HbA1c_str[1:-1].split(',')[-1])
        ind_v[:, 164] = true_HbA1c_float # flatten for test set
        H_bf_obs_wind.append(ind_v[0, 0])
    else:
        ind_v[:, 164] = 0.0 
        H_other.append(ind_v[0, 0])
    ###################
    # GFR
    ####################
    if sum(ind_v[:25, 168]) > 0: # HbA1c tested in obs window
        last_test_idx = 25 - 1- (list(ind_v[:25, 168])[::-1].index(1)) # 1st HbA1c test in obs window
        true_GFR_str = ind_v[last_test_idx, 189]
        true_GFR_float = float(true_GFR_str[1:-1].split(',')[-1]) #flatten for test set
        ind_v[last_test_idx:, 167] = true_GFR_float
        G_in_obs_wind.append(ind_v[0, 0])
    elif sum(ind_v[:25, 169]) > 0: # HbA1c tested prior to obs window and not in obs window
        true_GFR_str = ind_v[0, 189]
        true_GFR_float = float(true_GFR_str[1:-1].split(',')[-1])
        ind_v[:, 167] = true_GFR_float # flatten for test set
        G_bf_obs_wind.append(ind_v[0, 0])
    else:
        ind_v[:, 167] = 0.0
        G_other.append(ind_v[0, 0])
    test_v[(i*29):((i+1)*29), :] = ind_v
    if i%1000==0:
        print("Individual :", i)
        print(time.time() - start_time)
print("Total time:", time.time() - start_time)
print("HBA1C in obs window:", len(H_in_obs_wind))
print("HBA1C before obs window:", len(H_bf_obs_wind))
print("HBA1C set to 0.0:", len(H_other))
print("EGFR in obs window:", len(G_in_obs_wind))
print("EGFR before obs window:", len(G_bf_obs_wind))
print("EGFR set to 0.0:", len(G_other))

(281271, 190)
Number of inds: 9699
Individual : 0
0.0032625198364257812
Individual : 1000
2.6371500492095947
Individual : 2000
5.271834373474121
Individual : 3000
7.928622484207153
Individual : 4000
10.575850486755371
Individual : 5000
13.263536214828491
Individual : 6000
15.951281547546387
Individual : 7000
18.650800704956055
Individual : 8000
21.318427801132202
Individual : 9000
24.023045539855957
Total time: 25.868678092956543
HBA1C in obs window: 9234
HBA1C before obs window: 19
HBA1C set to 0.0: 446
EGFR in obs window: 9666
EGFR before obs window: 11
EGFR set to 0.0: 22


In [29]:
# data check
'''
HBA1C: 164
TEST_HBA1C: 165
TESTED_HBA1c: 166
EGFR: 167
TEST_EGFR: 168
TESTED_EGFR: 169
TRUE_HBA1C: 188
TRUE_EGFR: 189
'''    
test_df = pd.DataFrame(test_v)
test_df.columns = list(view_df.columns)
idx = 5
#id_id = H_in_obs_wind[idx]
#ind_id = H_bf_obs_wind[idx]
#ind_id = H_other[idx]
#ind_id = G_in_obs_wind[idx]
ind_id = G_bf_obs_wind[idx]
#ind_id = G_other[idx]
test_df[test_df.iloc[:, 0]==ind_id].\
iloc[:,[0,1,164,165,166,167,168,169,188,189]]

,VSIMPLE_INDEX_MASTER,QUARTER,HBA1C,TEST_HBA1C,TESTED_HBA1C,EGFR,TEST_EGFR,TESTED_EGFR,TRUE_HBA1C,TRUE_EGFR
63017,90050,0,40,0,0,71.55,0,1,[],[71.55]
63018,90050,1,40,0,0,71.55,0,1,[],[]
63019,90050,2,40,0,0,71.55,0,1,[],[]
63020,90050,3,40,0,0,71.55,0,1,[],[]
63021,90050,4,40,0,0,71.55,0,1,[],[]
63022,90050,5,40,0,0,71.55,0,1,[],[]
63023,90050,6,40,0,0,71.55,0,1,[],[]
63024,90050,7,40,0,0,71.55,0,1,[],[]
63025,90050,8,40,0,0,71.55,0,1,[],[]
63026,90050,9,40,0,0,71.55,0,1,[],[]


## impute HbA1c and eGFR

In [27]:
print(len(test_df[test_df['HBA1C']==0]['VSIMPLE_INDEX_MASTER'].unique()))
print(len(test_df[test_df['EGFR']==0]['VSIMPLE_INDEX_MASTER'].unique()))

446
22


In [33]:
print(train_v.shape)
train_df = pd.DataFrame(train_v)
train_df.columns = list(view_df.columns)

print(view_df['CVD_ISOSORBRIDE_DINITRATE'].unique())
view_df = view_df.drop(['CVD_ISOSORBRIDE_DINITRATE'], axis=1)
print(train_df['CVD_ISOSORBRIDE_DINITRATE'].unique())
train_df = train_df.drop(['CVD_ISOSORBRIDE_DINITRATE'], axis=1)

print(train_df.shape)
print(train_df[train_df['HBA1C']==0].shape)
print(train_df[train_df['EGFR']==0].shape)
print(test_df.shape)
print(test_df[test_df['HBA1C']==0].shape)
print(test_df[test_df['EGFR']==0].shape)

(2479500, 191)
[0.]
[0.0]
(2479500, 190)
(0, 190)
(0, 190)
(281271, 190)
(12934, 190)
(638, 190)


In [34]:
train_v = copy.deepcopy(train_df.values)

In [35]:
# linear model fit using 
# Age, sex, nzdep and ethnicity
# from training set
HBA1C_lm = LinearRegression().fit(train_v[:, [2,3,4,5,6,7,8,9]],
                                  train_v[:, 164])
GFR_lm = LinearRegression().fit(train_v[:, [2,3,4,5,6,7,8,9]],
                                 train_v[:, 167])
# impute data in test set
print(test_v.shape)
for i in range(test_v.shape[0]):
    # impute HbA1c and GFR in test set
    if test_v[i, 164] == 0:
        HBA1C_v = HBA1C_lm.predict(np.array([test_v[i, [2,3,4,5,6,7,8,9]]]))
        #print("HBA1C:", HBA1C_v[0])
        test_v[i, 164] = copy.deepcopy(HBA1C_v[0])
    if test_df.iloc[i, 167] == 0:
        GFR_v = GFR_lm.predict(np.array([test_v[i, [2,3,4,5,6,7,8,9]]]))
        #print("GFR:", GFR_v[0])
        test_v[i, 167] = copy.deepcopy(GFR_v[0])
    if i % 10000 == 0:
        print('Row:', i)
        print(time.time() - start_time)
print('Entire process took:', time.time()) 

(281271, 190)
Row: 0
1743.2668807506561
Row: 10000
1743.398525238037
Row: 20000
1743.4975872039795
Row: 30000
1743.6054294109344
Row: 40000
1743.7260403633118
Row: 50000
1743.839944601059
Row: 60000
1743.9618890285492
Row: 70000
1744.0818347930908
Row: 80000
1744.1924741268158
Row: 90000
1744.3112878799438
Row: 100000
1744.4236807823181
Row: 110000
1744.5435750484467
Row: 120000
1744.6643018722534
Row: 130000
1744.774407863617
Row: 140000
1744.8920412063599
Row: 150000
1745.0112109184265
Row: 160000
1745.126446723938
Row: 170000
1745.244605064392
Row: 180000
1745.3576474189758
Row: 190000
1745.4655129909515
Row: 200000
1745.5919468402863
Row: 210000
1745.7076163291931
Row: 220000
1745.8180718421936
Row: 230000
1745.9335029125214
Row: 240000
1746.0429241657257
Row: 250000
1746.1646835803986
Row: 260000
1746.2772798538208
Row: 270000
1746.3977682590485
Row: 280000
1746.5196659564972
Entire process took: 1600740528.3809412


In [36]:
test_df = pd.DataFrame(test_v)
test_df.columns = list(view_df.columns)

In [39]:
print(test_df.shape)
print(train_df.shape)
print(test_df.shape[0]/29)
print(train_df.shape[0]/29)

(281271, 190)
(2479500, 190)
9699.0
85500.0


In [40]:
feather.write_dataframe(test_df, "/home/whsu014/data/Post_PREDICT_Test_with_TRUE_9699inds.feather")
feather.write_dataframe(train_df, "/home/whsu014/data/Post_PREDICT_Train_with_TRUE_85500inds.feather")